# Notebook 2-2: HMM Post-Hoc Spectral Estimation

In this notebook, we will cover the following topics:
1. **Multitaper Spectral Estimation**: We will use the multitaper method to estimate the spectral characteristics of the HMM states.
2. **Non-negative Matrix Factorization (NNMF)**: We will apply NMF to the data to extract features that can be used for further analysis.


## 1. Multitaper Spectral Estimation
Here we will use the multitaper to estimate the state-specific power spectral density (PSD) and coherence. We will make use of the original data as well as the inferred state time courses. First, we need to load the original data and the inferred state time coruses.

In [ ]:
from glob import glob
import pickle
from osl_dynamics.data import Data

alpha = pickle.load(open("results/inf_params/alp.pkl", "rb"))
files = sorted(glob("wakeman-henson/*/*_sflip_lcmv-parc-raw.fif"))
data = Data(files, picks="misc", reject_by_annotation="omit")
trimmed_data = data.trim_time_series(n_embeddings=15, sequence_length=200)

Quick Q: Why do we need to trim the original data?

Quick sanity check to make sure the data and alpha has the same length.

In [ ]:
for d, a in zip(trimmed_data, alpha):
    print(d.shape, a.shape)

Next we perform the multitaper with the `multitaper_spectra` function in the `osl_dynamics.analysis.spectra` module.

In [ ]:
import os
import numpy as np
from osl_dynamics.analysis import spectral

f, psd, coh, w = spectral.multitaper_spectra(
    data=trimmed_data,
    alpha=alpha,
    sampling_frequency=250,
    frequency_range=[0.5, 45],
    return_weights=True,
)

spectra_dir = "results/spectra"
os.makedirs(spectra_dir, exist_ok=True)

np.save(f"{spectra_dir}/f.npy", f)
np.save(f"{spectra_dir}/psd.npy", psd)
np.save(f"{spectra_dir}/coh.npy", coh)
np.save(f"{spectra_dir}/w.npy", w)


Quick Q: What if you have lots of session? How do you handle that in the multitaper?

# 2. Non-negative Matrix Factorization (NNMF)
Next we will use the non-negative matrix factorisation (NNMF) to decompose the PSD into a set of basis functions. This is useful for visualising the spectral characteristics of each state.

In [ ]:
nnmf = spectral.decompose_spectra(coh, n_components=2)

np.save(f"{spectra_dir}/nnmf_2.npy", nnmf)